In [1]:
import os
os.chdir('../movies')
from movieLens import MovieLens

# Load the movie Lens class
ml = MovieLens()

# Algorithm

In [2]:
from surprise import Dataset, Reader
from surprise.model_selection import train_test_split
# from RBM import RBM

from surprise import accuracy
# import heapq
# from collections import defaultdict
# from operator import itemgetter
import numpy as np
import pandas as pd

In [3]:
# Load the ratings dataset
ratings = ml.ratings.copy()
ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [4]:
print(ratings['rating'].nunique())
list(ratings['rating'].unique())

10


[4.0, 5.0, 3.0, 2.0, 1.0, 4.5, 3.5, 2.5, 0.5, 1.5]

In [5]:
# Method from the Surprise library to load the DataFrame 
# Define the Reader object to parse the dataframe
reader = Reader(rating_scale=(ratings['rating'].min(), ratings['rating'].max()))

# Load the dataframe as a ratings dataset
ratingsDataset = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)

# Build the full trainset
trainSet, testSet = train_test_split(ratingsDataset, test_size=0.2, random_state=42)
antitest = trainSet.build_anti_testset()

# PRUEBA 1

Para preparar los datos de train y test, necesitamos crear conjuntos en formato de matriz con cada fila representando a un usuario y cada celda de la fila representando la valoración de cada película, dado que esta es la entrada esperada por el algoritmo RBM.

Para ello, necesitamos el número total de usuarios como números de fila y el número total de películas como número de columna
* num_users = trainSet.n_users
* num_movies = trainSet.n_items

In [ ]:
def fitData(trainset):
    #AlgoBase.fit(self, trainset)

    num_users = trainset.n_users
    num_movies = trainset.n_items
    
    # 3D matrix: users, movies and ratings
    # Ratings has size 10 given the possible rating values
    trainingMatrix = np.zeros([num_users, num_movies, 10], dtype=np.float32)

    for (uid, iid, rating) in trainset.all_ratings():
        adjustedRating = int(float(rating)*2.0) - 1
        trainingMatrix[int(uid), int(iid), adjustedRating] = 1

    # Flatten to a 2D array, with nodes for each possible rating type on each possible item, for every user.
    trainingMatrix = np.reshape(trainingMatrix, [trainingMatrix.shape[0], -1])
    
    return trainingMatrix

In [ ]:
trainingMatrix = fitData(trainSet)
trainingMatrix.shape

In [ ]:
from RBM import RBM

# Create an RBM with (num items * rating values) visible nodes
model = RBM(trainingMatrix.shape[1])
model.train(trainingMatrix)

In [ ]:
def getPredictions(rbm,num_users,num_movies,trainingMatrix):
    
    predictedRatings = np.zeros([num_users, num_movies], dtype=np.float32)
    
    for uiid in range(num_users):
        if (uiid % 50 == 0):
            print("Processing user ", uiid)
        recs = rbm.getRecommendations([trainingMatrix[uiid]])
        recs = np.reshape(recs, [num_movies, 10])

        for itemID, rec in enumerate(recs):
            # The obvious thing would be to just take the rating with the highest score:                
            #rating = rec.argmax()
            # ... but this just leads to a huge multi-way tie for 5-star predictions.
            # The paper suggests performing normalization over K values to get probabilities
            # and take the expectation as your prediction, so we'll do that instead:

            normalized = np.exp(rec)/np.sum(np.exp(rec), axis=0)
            rating = np.average(np.arange(10), weights=normalized)
            
            predictedRatings[uiid, itemID] = (rating + 1) * 0.5

    return predictedRatings

In [ ]:
predictions = getPredictions(model,trainSet.n_users,trainSet.n_items,trainingMatrix)

In [ ]:
predictions

In [ ]:
rmse = accuracy.rmse(predictions)
mae = accuracy.mae(predictions)

# Prueba 2

In [ ]:
from RBM_v2 import RBM_v2

# Create an RBM with (num items * rating values) visible nodes
model2 = RBM_v2(trainingMatrix.shape[1])
model2.train(trainingMatrix)

In [ ]:
predictions = getPredictions(model2,trainSet.n_users,trainSet.n_items,trainingMatrix)

In [ ]:
predictions

In [ ]:
model2.test(testSet)

In [ ]:
from surprise.model_selection import train_test_split

# Load the dataframe as a ratings dataset
ratingsDataset = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)

# Split the dataset into train and test sets
trainSet, testSet = train_test_split(ratingsDataset, test_size=0.2, random_state=42)

In [ ]:
model2.test(testSet)

In [ ]:
from surprise import accuracy
from surprise import Prediction

# Assuming `testSet` is the list of tuples

# Convert the testSet tuples into Prediction objects
prediction_objects = [Prediction(uid=t[0], iid=t[1], r_ui=t[2], est=predictions[i], details={})
                      for i, t in enumerate(testSet)]

# Calculate RMSE and MAE
rmse = accuracy.rmse(prediction_objects)
mae = accuracy.mae(prediction_objects)


# Probar si con RBM AlgotBase se pueden sacar las métricas

In [ ]:
from RBMAlgorithm import RBMAlgorithm
RBM = RBMAlgorithm(epochs=20)

In [ ]:
RBM.test(testSet)

# Metrics

In [6]:
import os
os.chdir('../metrics')
from metrics import evaluationMetrics
em = evaluationMetrics()

In [ ]:
# Get test and antitest predictions
predtest = model.test(testSet)
predantitest = model.test(antitest)

# Get top N recommended movies for each user based on estimated ratings
top_10_RBM = em.getTopN(predantitest,minimumRating = 3.5)

## Métricas de precisión: RMSE y MAE

In [ ]:
# # RMSE
rmse = accuracy.rmse(predtest)

# MAE
mae = accuracy.mae(predtest)

# rmse = accuracy.rmse(predictions)
# mae = accuracy.mae(predictions)

## Métricas de relevancia: Precision, Recall y NDCG

In [ ]:
# Precision
precisions = em.getPrecision(predtest, k=10, threshold=3.5)

# Mean Average Precision
mapModel = np.mean(list(precisions.values()))

# Recall
recalls = em.getRecall(predtest, k=10, threshold=3.5)

# Mean Average Recall
marModel = np.mean(list(recalls.values()))

# Normalized discounted cummulative gain (NDCG)
ndcgs, mean_ndcg = em.getNDCG(predtest,10)

## Otras métricas de interés: Coverage, User Coverage y Novelty

In [ ]:
# Coverage
coverage = em.getCoverage(top_10_RBM,trainSet.n_items,trainSet.all_users())

# User coverage
user_coverage = em.getUserCoverage(top_10_RBM, trainSet.n_users,4)

# Novelty
novelty = em.getNovelty(top_10_RBM,trainSet)

Por último creamos un dataframe con todas las métricas de evaluación asociadas al modelo

In [ ]:
cols = ["Model","RMSE","MAE","MAP","MAR","Mean_NDCG","Coverage","User_Coverage","Novelty"]
metrics_data = []

# Append the results to the list of dictionaries
metrics_data.append({"Model": "RBM", "RMSE": rmse, "MAE": mae, "MAP": mapModel, "MAR": marModel,
                     "Mean_NDCG": mean_ndcg, "Coverage": coverage, "User_Coverage": user_coverage,
                     "Novelty": novelty})

# Convert the list of dictionaries into a DataFrame
metrics_df = pd.DataFrame(metrics_data, columns=cols)
metrics_df

# Random Algorithm

Vamos a evaluar también un modelo Random, en concreto "NormalPredictor" para poder comparar sus resultados con el resto de modelos. NormalPredictor es un algoritmo simple en Surprise que predice calificaciones aleatoriamente basado en la distribución del conjunto de entrenamiento. Supone una distribución normal de las calificaciones y genera predicciones aleatorias en función de esa distribución.

In [7]:
from surprise import NormalPredictor

# Create the model
Random = NormalPredictor()
Random.fit(trainSet)

In [ ]:
# Get test and antitest predictions
predtest_random = Random.test(testSet)
predantitest_random = Random.test(antitest)

# Get top N recommended movies for each user based on estimated ratings
top_10_random = em.getTopN(predantitest_random,minimumRating = 3.5)

## Compute metrics

In [ ]:
# Accuracy Metrics
rmse_random = accuracy.rmse(predtest_random)
mae_random = accuracy.mae(predtest_random)

# Relevance metrics
precisions_random = em.getPrecision(predtest_random, k=10, threshold=3.5)
mapModel_random = np.mean(list(precisions_random.values()))

recalls_random = em.getRecall(predtest_random, k=10, threshold=3.5)
marModel_random = np.mean(list(recalls_random.values()))

ndcgs_random, mean_ndcg_random = em.getNDCG(predtest_random,10)

# Other metrics
coverage_random = em.getCoverage(top_10_random,trainSet.n_items,trainSet.all_users())
user_coverage_random = em.getUserCoverage(top_10_random, trainSet.n_users,4)
novelty_random = em.getNovelty(top_10_random,trainSet)

In [ ]:
cols = ["Model","RMSE","MAE","MAP","MAR","Mean_NDCG","Coverage","User_Coverage","Novelty"]
metrics_data = []

# Append the results to the list of dictionaries
metrics_data.append({"Model": "random", "RMSE": rmse_random, "MAE": mae_random, "MAP": mapModel_random, "MAR": marModel_random,
                     "Mean_NDCG": mean_ndcg_random, "Coverage": coverage_random, "User_Coverage": user_coverage_random,
                     "Novelty": novelty_random})

# Convert the list of dictionaries into a DataFrame
metrics_df = pd.DataFrame(metrics_data, columns=cols)
metrics_df